# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 11
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr11.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr11.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 387 387

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000177951,ENSG00000142082,ENSG00000185627,ENSG00000185201,ENSG00000185885,ENSG00000142089,ENSG00000185187,ENSG00000174915,ENSG00000174775,ENSG00000185507
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000177951,1.00000000,0.06467838,0.01383004,0.01835632,-0.1160407,0.09404034,0.1771446168,-0.040395376,-0.04674757,0.04000345
ENSG00000142082,0.06467838,1.00000000,-0.15957186,-0.06578159,0.1416064,-0.14939870,0.0405351920,-0.000672903,0.08710930,-0.10074523
ENSG00000185627,0.01383004,-0.15957186,1.00000000,0.05390432,0.0385715,-0.01263637,0.0002437382,-0.216339443,-0.06214113,0.05229397
ENSG00000185201,0.01835632,-0.06578159,0.05390432,1.00000000,-0.0618631,0.06515788,0.0835571610,0.021877127,0.02263662,0.07407167
ENSG00000185885,-0.11604075,0.14160642,0.03857150,-0.06186310,1.0000000,-0.18196319,0.1008124295,-0.032441819,-0.03839574,-0.21432607
ENSG00000142089,0.09404034,-0.14939870,-0.01263637,0.06515788,-0.1819632,1.00000000,-0.1382320098,0.048044527,0.02692348,0.04434910


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 48

[1] -0.01499434 -0.02869884 -0.03137953 -0.06560003 -0.08089972 -0.08483895
 [7] -0.13275628 -0.13642752 -0.15383348 -0.16167530 -0.16347197 -0.18176226
[13] -0.23347464 -0.25480104 -0.27257514 -0.28397233 -0.29115765 -0.31453658
[19] -0.38454127 -0.41786939 -0.42913670 -0.44633083 -0.50456793 -0.50868032
[25] -0.53770990 -0.55502076 -0.58318616 -0.66636122 -0.70790967 -0.85718685
[31] -0.87301407 -0.92620202 -1.01531935 -1.04905386 -1.08818142 -1.16578277
[37] -1.31543680 -1.38041631 -1.39997148 -1.62604369 -1.80877901 -1.87290735
[43] -2.03586321 -2.07419467 -2.15977552 -2.50014489 -2.92795069 -4.35306094

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 387 387

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000177951,ENSG00000142082,ENSG00000185627,ENSG00000185201,ENSG00000185885,ENSG00000142089,ENSG00000185187,ENSG00000174915,ENSG00000174775,ENSG00000185507
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000177951,1.00000000,0.06467838,0.01383004,0.01835632,-0.1160407,0.09404034,0.1771446168,-0.040395376,-0.04674757,0.04000345
ENSG00000142082,0.06467838,1.00000000,-0.15957186,-0.06578159,0.1416064,-0.14939870,0.0405351920,-0.000672903,0.08710930,-0.10074523
ENSG00000185627,0.01383004,-0.15957186,1.00000000,0.05390432,0.0385715,-0.01263637,0.0002437382,-0.216339443,-0.06214113,0.05229397
ENSG00000185201,0.01835632,-0.06578159,0.05390432,1.00000000,-0.0618631,0.06515788,0.0835571610,0.021877127,0.02263662,0.07407167
ENSG00000185885,-0.11604075,0.14160642,0.03857150,-0.06186310,1.0000000,-0.18196319,0.1008124295,-0.032441819,-0.03839574,-0.21432607
ENSG00000142089,0.09404034,-0.14939870,-0.01263637,0.06515788,-0.1819632,1.00000000,-0.1382320098,0.048044527,0.02692348,0.04434910


,ENSG00000177951,ENSG00000142082,ENSG00000185627,ENSG00000185201,ENSG00000185885,ENSG00000142089,ENSG00000185187,ENSG00000174915,ENSG00000174775,ENSG00000185507
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000177951,1.000000000,-0.00285156,0.005834611,-0.00328707,-0.07672903,0.05717433,0.07990477,-0.002244259,-0.02203210,0.02809111
ENSG00000142082,-0.002851560,1.00000000,-0.112784503,-0.08709298,0.07196395,-0.04865865,0.04776076,0.022990948,0.02298238,-0.06570505
ENSG00000185627,0.005834611,-0.11278450,1.000000000,0.11452384,0.03570503,-0.03518592,-0.05159305,-0.076258156,-0.04612396,0.04722397
ENSG00000185201,-0.003287070,-0.08709298,0.114523837,1.00000000,0.04785150,-0.06080794,-0.04139225,0.026716669,0.03061249,0.01449240
ENSG00000185885,-0.076729032,0.07196395,0.035705029,0.04785150,1.00000000,-0.15816945,0.01753376,-0.017072725,-0.01337802,-0.11184699
ENSG00000142089,0.057174326,-0.04865865,-0.035185924,-0.06080794,-0.15816945,1.00000000,-0.01414340,0.013979248,0.01947357,0.05973707


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)